In [1]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("simplescaling/s1.1-7B")

embed_weights: torch.Tensor = model.model.embed_tokens.weight
unembed_weights: torch.Tensor = model.lm_head.weight

print("embed_tokens.weight shape:", embed_weights.shape)

/root/ai2/tokens/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:00<00:00, 169.57it/s]

embed_tokens.weight shape: torch.Size([152064, 3584])


In [2]:
embed_weights

Parameter containing:
tensor([[-1.5497e-02, -4.1463e-03,  1.4529e-02,  ...,  1.0730e-02,
          4.1811e-02, -1.7753e-02],
        [ 1.5380e-02,  1.5618e-02,  1.7398e-02,  ..., -1.9151e-02,
          1.6571e-02,  7.5589e-03],
        [-8.4275e-03, -4.7465e-03,  4.5070e-03,  ..., -1.0826e-03,
         -2.7765e-02,  7.6024e-03],
        ...,
        [-1.1755e-37,  1.1755e-37,  1.1755e-37,  ..., -1.1755e-37,
          1.1755e-37,  1.1755e-37],
        [ 1.1755e-37, -1.1755e-37,  1.1755e-37,  ..., -1.1755e-37,
          1.1755e-37, -1.1755e-37],
        [ 1.1755e-37, -1.1755e-37, -1.1755e-37,  ...,  1.1755e-37,
          1.1755e-37, -1.1755e-37]], requires_grad=True)

In [3]:
# Idea (doesn't work):
    # model.embed_tokens.weight // lm_head.weight
        # [152 064, 3 584] tensor
        # we learn a [XXXXXX, 3 584] tensor which should be the same
        # minimize L2 loss

    # Then, we swap the embedding layer and re-run the model

In [4]:
# The *only* way to learn this is by replacing the tokenizer and embedding layers, then continually re-training the embedding layers with the *entire* model

# This is because the sequence lengths will change between tokenizers, so the only standard signal would be the text itself (not the next-token distribution, or the hidden representations)

# You could re-run SFT (~1B-10B toks?) or mid-training (50B-300B toks) with the swapped model